In [1]:
import os
import pandas as pd
import geopandas as gpd
import pygeos as pg
import numpy as np
import tensorflow as tf
import sqlalchemy as sq
import calendar
from dotenv import load_dotenv
from IPython.display import clear_output
from matplotlib import pyplot as plt
from DataService import DataService

2022-11-26 19:56:52.314200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 19:56:52.452991: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
os.chdir('/tf')
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')
NULLFLAG = -9999

In [3]:
# Connect to the database
pullService = DataService(PGDB, PGUSER, PGPW)
db_pull_con = pullService.connect()

pushService = DataService(PGDB, PGUSER, PGPW)
db_push_con = pushService.connect()

In [4]:
# Load the data
weatherDailyAggTable = "WeatherDataHourlyAggDaily"
query = "SELECT * FROM public.\"{}\";".format(weatherDailyAggTable)
dfAgg = pd.read_sql(query, db_pull_con)

In [5]:
# Load fire data
fireTable = "lgFireFifty"
query = "SELECT * FROM public.\"{}\" WHERE \"YEAR\" > 2009;".format(fireTable)
dfFire = gpd.GeoDataFrame.from_postgis(query, db_pull_con)

In [6]:
centroids = "lgFireFiftyCentroids"
query = "SELECT * FROM public.\"{}\";".format(centroids)
dfCent = pd.read_sql(query, db_pull_con)

In [7]:
dfAgg.describe()

,Year,Month,Day,MeanTemp,MinTemp,MaxTemp,MeanDewPoint,MinDewPoint,MaxDewPoint,MeanHumidity,MinHumidity,MaxHumidity,MeanPressure,MinPressure,MaxPressure,MeanWindSpeed,MinWindSpeed,MaxWindSpeed,MeanWindChill,MinWindChill,MaxWindChill,TotalPrecip,MeanWindDirection
count,72790.0,72790.0,72790.0,72790.0,72790.0,72790.0,71852.0,71852.0,71852.0,71852.0,71852.0,71852.0,71487.0,71487.0,71487.0,71487.0,71487.0,71487.0,71487.0,71487.0,71487.0,72790.0,71487.0
mean,2015.7,6.5,15.7,-0.4,-5.7,4.4,-5.6,-9.1,-2.3,71.4,55.2,86.8,97.4,97.0,97.7,10.2,3.4,18.2,-18.9,-22.1,-14.6,1.0,18.5
std,3.4,3.5,8.8,14.8,14.7,15.3,13.4,14.4,12.7,13.5,19.5,9.4,1.3,1.3,1.3,4.8,3.6,7.4,6.5,7.6,7.8,17.2,7.4
min,2010.0,1.0,1.0,-42.6,-48.5,-36.2,-47.3,-54.9,-41.2,17.8,6.0,28.0,92.4,91.7,92.7,0.0,0.0,0.0,-53.3,-60.0,-49.0,0.0,0.0
25%,2013.0,4.0,8.0,-11.9,-17.0,-7.6,-15.1,-19.5,-11.1,62.5,39.0,81.0,96.5,96.1,96.8,6.8,0.0,13.0,-21.0,-24.0,-19.6,0.0,12.5
50%,2016.0,7.0,16.0,1.3,-2.8,5.3,-3.8,-6.5,-1.1,72.9,56.0,89.0,97.3,96.9,97.6,9.5,2.0,17.0,-18.1,-19.6,-17.0,0.0,18.4
75%,2019.0,10.0,23.0,12.8,6.9,17.8,6.0,2.9,8.8,81.7,71.0,94.0,98.2,97.9,98.6,12.9,5.0,22.0,-15.2,-17.5,-7.0,0.2,24.5
max,2021.0,12.0,31.0,31.5,30.2,39.9,19.5,17.3,30.0,100.0,100.0,100.0,102.2,102.0,102.4,50.3,34.0,93.0,-2.1,-4.0,0.0,2812.5,36.0


In [8]:
dfFire.describe()

,EntryID,YEAR,MONTH,DAY,SIZE_HA,CALC_HA
count,3265.0,3265.0,3265.0,3265.0,3265.0,3265.0
mean,6282.6,2014.5,6.7,16.1,8200.9,8125.4
std,3983.5,2.8,1.1,9.2,32210.4,31947.1
min,1.0,2010.0,2.0,1.0,200.0,36.1
25%,3056.0,2012.0,6.0,7.0,487.0,482.5
50%,5708.0,2015.0,7.0,16.0,1375.7,1370.4
75%,9085.0,2017.0,7.0,25.0,4719.8,4664.7
max,12827.0,2020.0,12.0,31.0,596459.1,596459.1
